In [1]:
import os
import cv2
import numpy as np
import glob

# Import of Metavision Machine Learning binding
import metavision_sdk_ml
import metavision_sdk_cv
from metavision_sdk_core import EventBbox
from metavision_sdk_core import BaseFrameGenerationAlgorithm
from metavision_core.event_io.py_reader import EventDatReader
from metavision_core.event_io.events_iterator import EventsIterator
from metavision_core.event_io import EventNpyReader

%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [11, 7]

In [2]:
hsv = np.uint8([[[0,255,255], [20,255,255], [40,255,255], [60,255,255], [80,255,255], [100,255,255], [120,255,255]]])
bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR).squeeze()
print(bgr)

[[  0   0 255]
 [  0 170 255]
 [  0 255 170]
 [  0 255   0]
 [170 255   0]
 [255 170   0]
 [255   0   0]]


In [ ]:
event_file = "17-04-11_15-13-23_122500000_182500000"
event_dat_path = "/home/tkyen/opencv_practice/data_3/Gen1_Automotive/detection_dataset_duration_60s_ratio_1.0/test/{}_td.dat".format(event_file)
event_npy_path = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_evt/{}_td.npy".format(event_file)
gt_label_path = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/{}_bbox.npy".format(event_file)

video_dir = "./video_ev_gt"
os.makedirs(video_dir, exist_ok=True)

In [20]:
event_dat = EventDatReader(event_dat_path)
event_dat.seek_time(0)
ev_height, ev_width = event_dat.get_size()

events_all = np.load(event_npy_path)
labels_all = np.load(gt_label_path)
delta_t = 5000

# VideoWriter
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
frame_rate = 10
video_writer = cv2.VideoWriter(os.path.join(video_dir, '{}.mp4'.format(event_file)), fourcc, frame_rate, (ev_width, ev_height))
colors = [[0,0,255], [0,255,0]]
ev_dtype = {'names':['x','y','p','t'], 'formats':['<u2','<u2','<i2','<i8'], 'offsets':[0,2,4,8], 'itemsize':16}

for i in range(int(60e6//delta_t)):    
    evts = events_all[(events_all['t']>=i*delta_t) * (events_all['t']<(i+1)*delta_t)]
    evts_t = evts['t']
    evts_x = evts['x']
    evts_y = evts['y']
    evts_p = evts['p']
    evts = np.concatenate((evts_x[:,None], evts_y[:,None], evts_p[:,None], evts_t[:,None]), axis=1)
    evts_list = evts.tolist()
    evts_list = list(map(tuple, evts_list))
    events = np.empty((0,4), dtype=ev_dtype)
    events = np.append(events, np.asarray(evts_list, dtype=ev_dtype))

    image_all = np.zeros((ev_height, ev_width, 3), dtype=np.uint8)
    BaseFrameGenerationAlgorithm.generate_frame(events, image_all)

    cv2.putText(image_all, '{} ms / {} ms'.format(i*delta_t//1000, (i+1)*delta_t//1000), 
                (ev_width-200, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(image_all, 'GT',
                (10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

    gt_boxes = labels_all[(labels_all['t']>=i*delta_t) * (labels_all['t']<(i+1)*delta_t)]
    for gt_box in gt_boxes:
        t, x, y, w, h, class_id, confidence, track_id, invalid = gt_box
        x, y, w, h = int(x), int(y), int(w), int(h)
        if invalid == False:
            cv2.rectangle(image_all, (x, y), (x+w, y+h), colors[class_id], 1)
    video_writer.write(image_all)

video_writer.release()

[(99999, 40., 144., 52., 33., 0, 1., 202, False)]


In [35]:
def nms(box_events, scores, iou_thresh=0.5):
    """NMS on box_events

    Args:
        box_events (np.ndarray): nx1 with dtype EventBbox, the sorting order of those box is used as a
            a criterion for the nms.
        scores (np.ndarray): nx1 dtype of plain dtype, needs to be argsortable.
        iou_thresh (float): if two boxes overlap with more than `iou_thresh` (intersection over union threshold)
            with each other, only the one with the highest criterion value is kept.

    Returns:
        keep (np.ndarray): Indices of the box to keep in the input array.
    """
    x1 = box_events['x']
    y1 = box_events['y']
    x2 = box_events['x'] + box_events['w']
    y2 = box_events['y'] + box_events['h']

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= iou_thresh)[0]
        order = order[inds + 1]

    return sorted(keep)

In [31]:
event_dat_dir = "/home/tkyen/opencv_practice/data_3/Gen1_Automotive/detection_dataset_duration_60s_ratio_1.0/test"
event_npy_dir = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_evt"
gt_label_dir = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl"
dt_label_dir = "/home/tkyen/opencv_practice/HMNet/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test_pretrained"
video_dir = dt_label_dir + "_video"
os.makedirs(video_dir, exist_ok=True)

event_dat_list = sorted(glob.glob(os.path.join(event_dat_dir, "*.dat")))
event_npy_list = sorted(glob.glob(os.path.join(event_npy_dir, "*.npy")))
gt_label_list = sorted(glob.glob(os.path.join(gt_label_dir, "*.npy")))
dt_label_list = sorted(glob.glob(os.path.join(dt_label_dir, "*.npy")))

ev_dtype = {'names':['x','y','p','t'], 'formats':['<u2','<u2','<i2','<i8'], 'offsets':[0,2,4,8], 'itemsize':16}

In [ ]:
for i, (event_dat_path, event_npy_path, gt_label_path, dt_label_path) in enumerate(zip(event_dat_list, event_npy_list, gt_label_list, dt_label_list)):
    event_dat_file = os.path.basename(event_dat_path).replace('_td.dat', '')
    event_npy_file = os.path.basename(event_npy_path).replace('_td.npy', '')
    gt_file = os.path.basename(gt_label_path).replace('_bbox.npy', '')
    dt_file = os.path.basename(dt_label_path).replace('_bbox.npy', '')
    assert event_dat_file==event_npy_file==gt_file==dt_file
    # if event_npy_file != "17-04-13_15-05-43_1098500000_1158500000":
    #     continue

    event_dat = EventDatReader(event_dat_path)
    event_dat.seek_time(0)
    ev_height, ev_width = event_dat.get_size()

    event_npy = np.load(event_npy_path)

    gt_label = EventNpyReader(gt_label_path)
    gt_label.seek_time(0)

    dt_label = EventNpyReader(dt_label_path)
    dt_label.seek_time(0)

    delta_t = 50000

    # VideoWriter
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    frame_rate = 5
    video_writer = cv2.VideoWriter(os.path.join(video_dir, '{}.mp4'.format(event_dat_file)), fourcc, frame_rate, (ev_width*2, ev_height))
    colors = [[0,0,255], [0,255,0]]

    for i in range(int(60e6//delta_t)):
        evts = event_npy[(event_npy['t']>=delta_t*i)*(event_npy['t']<delta_t*(i+1))]
        evts_t = evts['t']
        evts_x = evts['x']
        evts_y = evts['y']
        evts_p = evts['p']
        evts = np.concatenate((evts_x[:,None], evts_y[:,None], evts_p[:,None], evts_t[:,None]), axis=1)
        evts_list = evts.tolist()
        evts_list = list(map(tuple, evts_list))
        events = np.empty((0,4), dtype=ev_dtype)
        events = np.append(events, np.asarray(evts_list, dtype=ev_dtype))
    
        gt_boxes = gt_label.load_delta_t(delta_t=delta_t)
        dt_boxes = dt_label.load_delta_t(delta_t=delta_t)
        dt_boxes = dt_boxes[nms(dt_boxes, dt_boxes['class_confidence'], iou_thresh=0.5)]

        if len(gt_boxes) > 0:
            assert gt_boxes['t'][0]>=delta_t*i and gt_boxes['t'][-1]<delta_t*(i+1)
        if len(dt_boxes) > 0:
            assert dt_boxes['t'][0]>=delta_t*i and dt_boxes['t'][-1]<delta_t*(i+1)

        image_all = np.zeros((ev_height, ev_width*2, 3), dtype=np.uint8)
        BaseFrameGenerationAlgorithm.generate_frame(events, image_all[:,:ev_width, :])
        image_all[:,ev_width:, :] = image_all[:,:ev_width, :].copy()
        cv2.putText(image_all, '{} ms / {} ms'.format(i*delta_t//1000, (i+1)*delta_t//1000), 
                    (ev_width-200, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image_all, 'GT',
                    (10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image_all, 'DT',
                    (ev_width+10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        image_all = cv2.line(image_all, (ev_width,0), (ev_width,ev_height), (0,255,255), 1)

        for gt_box in gt_boxes:
            t, x, y, w, h, class_id, confidence, track_id, invalid = gt_box
            x, y, w, h = int(x), int(y), int(w), int(h)
            if invalid == False:
                cv2.rectangle(image_all, (x, y), (x+w, y+h), colors[class_id], 1)

        for dt_box in dt_boxes:
            t, x, y, w, h, class_id, track_id, class_confidence = dt_box
            if class_confidence > 0.1:
                x, y, w, h = int(x), int(y), int(w), int(h)
                cv2.rectangle(image_all, (ev_width+x, y), (ev_width+x+w, y+h), colors[class_id], 1)

        video_writer.write(image_all)
    video_writer.release()

/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_24350